In [5]:
%run -i ../python/common.py

# The Shell - Part I: Having an ASCII conversation with the OS


As we read in the Unix [introduction](../unix/intro.ipynb) a key feature of Unix is its command line interface that was developed for [ASCII terminals](../unix/terminal.ipynb) devices. Our goal in this chapter is to understand the general model of the shell and how to start working in the terminal environment.  If you have not done so be sure to have read the sections on the Unix [Kernel](../unix/intro.ipynb#UnixKernel_sec), [User Programs](../unix/intro.ipynb#UnixUser_sec), how we [visualize a running Unix system](../unix/intro.ipynb#UnixViz_sec), the introduction to [Terminal Emulators](../unix/terminal.ipynb#TerminalEmulators_sec) and the chapter introducing [Files and Directories](../unix/files.ipynb).

> <img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/fyi.svg"> <p style="background-color:powderblue;"> It is a good idea to read this chapter with a terminal connection open so that you can explore the material as you read it.  Instructions and guidance on how to do this can be found in [Terminal Emulators](../unix/terminal.ipynb#TerminalEmulators_sec).

## Our interface to the Kernel

<img style="float: right; margin: 0px 0px 0px 10px;" width="50%" src="../images/UnixL01_SHCHT/04SHLLChat.png">

As discussed the core functionality of the Unix operating system is implemented by the Kernel.  But the kernel is really only responsible for making it easier to run and write application/user programs. The Kernel does not have any support for humans to directly interact with it.  That is where the shell comes in. The shell acts as our primary point of contact/interface not only to the kernel but also all the other programs installed.  

## Bash
Over the years there have been many variants of the shell program that have been developed not only for Unix but other operating systems as well (see [comparison of shells](https://en.wikipedia.org/wiki/Comparison_of_command_shells).   While there are differences between the various Unix shell programs the basic model of interaction we cover in this chapter is largely the same across all of them.   However, with respect to the exact syntax and details we will be focus on the [Bourne Again Shell (BASH)](https://en.wikipedia.org/wiki/Bash_(Unix_shell)),  as it is the default shell for Linux which is the version of Unix we use through out this book.

It is a Unix tradition that programs installed on the system, like Bash, include documentation on how it is used.  To access the manual pages you use the `man` program eg. `man man` would bring up the documentation about the manual itself.  So `man bash` will bring up the manual page for the bash shell.  You can find detailed coverage of the syntax and how bash works in the manual.  After we cover some basics it will be much easier to read and understand the Bash manual page as it assumes that you understand some basic Unix and generic shell concepts.  

## Shell Session
Remembering that ASCII terminals were the original devices created for humans to interactively work with computers the shell is designed to managed and communicate with a human at a terminal. Thus the human and the shell are really exchanging bytes that encoded information in ASCII.  For every new terminal connection software directs the kernel to start an instance of a shell program  to interact with a user at the terminal.  We consider this the be the start of a shell "session".  The session ends when either the communication between the shell and the terminal is disconnected or because the user purposefully exits the shell.  


### Terminal Windows and the Shell
Today of course, we rarely use physical terminals any more.  Rather, on our physical client computers (eg. laptops, desktops, tablets, etc) we can start many terminal emulator windows to establish a shell session for each (see below). Each session will stay active until we exit or close the session and kill the associated shell.  Since each window is connected to its own shell they represent independent "conversations" in which we can do different things concurrently.

In [6]:
display(HTML(htmlFig(
    [[
        {'src':"../images/UnixL01_SHCHT/041SHLLChat.png",
         'cellwidth': '47.5%'},
        {'src':"../images/terminalwins.png",
         'cellwidth': '52%'}
     ]],
    caption="Figure: Today terminal emulators running on our desktop act act as ASCII terminals. Every new window is a new terminal connection."
)))

,


> <img style="display:inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/do.svg"> <p style="background-color:lightgreen;">  While you are learning how to work with bash it can be really useful to keep a terminal open in which you keep the bash manual page open while you use another terminal shell session to do your exploring.

## Line oriented conversation

The way we interact with the shell can be thought of as a structured, interactive, "conversation". The conversation is a series of back and forth exchanges. We, the user, type in a request in the form of a "command line" and in response the shell takes some action and generate a particular response, as byte values, which are sent back to the terminal.  The terminal translates the response as ASCII characters displaying/printing the appropriate images to the screen so we can read it.   

### Lines

Given our use of ASCII what constitutes a line, in Unix, is very precise. It is a series of byte values terminated by a single 'new-line/line-feed' value.  Using an ASCII  table we see that the byte value for the new-line character is `0b10000010` in binary or expressed more concisely in hex as `0xA`. The common short hand notation for the newline byte value is `\n`.  Pressing the *return*  or *enter* key, depending on your keyboard, within a terminal window will generate this value.   We will simply refer to the key that generates a '\n' as the *enter* key.  

When `\n` is received by the shell it will begin working on processing the line as a command.  In this way it is line oriented.  If the user presses `\n` with no other preceding characters this is considered a blank or empty line.  

In the other direction, the terminal emulator is configured to take the appropriate action when it receives a `\n`.  That is to say it moves the cursor to the next lower position in the window (scrolling if configured to do so) and moves it to the left edge of the new line (again scrolling the window if configured to).


    
><img style="display:inline; margin: 1em 1em 0px 0px;" align="left" width="60" src="../images/fyi.svg"> <p style="background-color:powderblue;"> There are two basic modes that UNIX can use when sending bytes sent from the terminal connection to the program running on the connection (so far in our case this is the shell later we will see that programs that we run from the shell can be allowed to take over this role).  The two modes are character and line.  In character mode the terminal data is sent along to the program running as soon as they are received, including the `\n`.  In line (or buffered mode) the kernel stores up the characters and sent until it receives a `\n`.   At which point it sends the complete line with the `\n` to the program running. The default mode is typically line mode.  For the most part it does not really matter to us as the shell's behavior is largely the same as it is a line oriented program that is designed to process whole lines at a time.  However, later on this can be a source of confusion when we are writing programs that are not line oriented, rather they assume that they will see characters as soon as keys are pressed by the user.  



    
### The Prompt

To visually let us know that the shell is ready for us to send it a line it sends a configurable sequence of characters which we call the prompt string or simply the prompt.  When a new connection is established it sends the prompt to the terminal so that we know that it is ready for the session/conversation to begin.  The classic prompt string is often one either the dollar symbol, `$` or greater than sign `>` followed by a single space character.  In our illustrations and the default configuration we use set up the prompt to be the classic dollar sign followed by a space.

><img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="60" src="../images/fyi.svg"> <p style="background-color:powderblue;"> Today most system default to much more complicated prompt strings.  Where the prompt might state your user name, the date or time, what your current working directory is, etc.   Later when we learn about shell variables we will find out that there is a special variable `PS1` who's value is used to generate the prompt string every time the shell want to send it.  Given that every terminal session/window is connected to separate shells can be very useful to set each one with a unique prompt to help keep you organized eg. `export PS1='Term 1> '`.
If you are interested in the details use the following `man bash` to pull up the manual page for bash and look for the section called "Prompting".    


## Our first shell session

Before we more formally explain the operation of the shell lets poke it and see how it behaves.
Perhaps the simplest thing we can do is send it an empty line by pressing *enter* on its own.
What we should find is that the terminal will move to the next line and display the prompt again. 
But we want to be a little more careful to think about what happened behind the scenes so that we can build a more complete model in our mind about how the shell and terminal interactions work.

In [23]:
display(HTML(htmlFig(
    [
        [
            {'src':"../images/UnixL01_SHCHT/06SHLLChat.png",
             'caption':'A: Press Enter', 
             'border': '1px solid black',
             'padding':'1px'
            },
            {'src':"../images/UnixL01_SHCHT/07SHLLChat.png",
             'caption':'B: Shell "blank line" input processing' , 
             'border': '1px solid black',
             'padding':'1px'
            },
            {'src':"../images/UnixL01_SHCHT/08SHLLChat.png",
             'caption':'C: Shell sends Prompt back', 
             'border': '1px solid black',
             'padding':'1px'
            },
        ]
    ],
    id="fig:shell-blankline",
    caption="<center> Figure: Shell blank line behavior </center>"
)))

A: Press Enter,"B: Shell ""blank line"" input processing",C: Shell sends Prompt back


What happened was that after we pressed and release the *enter* key the terminal sent through the Unix kernel a byte to the instance of the shell program that our terminal is virtually connected too.  Precisely it would have sent the binary pattern of `0b10000010` or in hex `0xa` as the left most figure (A), above, illustrates.  The figure show both the `\n` human readable ASCII symbol and the underlying byte value.  In other diagrams we might stop showing the byte values and only show the ASCII human readable symbols but it is important to remember that under the covers the terminal and shell are really just exchanging raw binary values that use the ASCII standard to encode the data.  

The middle figure (B), above, cartoons the logic that the shell did in response to receiving the blank line.  It is programmed to do nothing and then simply send back to the terminal the prompt so that the user knows it is ready for another command line to be sent.  If you are interested in why the terminal displayed a `\n` causing the prompt sent by the shell to be on the next line of the terminal read the fyi ECHO box below.

><img   style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="50" src="../images/fyi.svg"> <p style="background-color:powderblue;"> **ECHO:**.  You might be wondering why is it that you see the characters you type at the keyboard of terminal appear on the screen of the terminal.  If pressing a key really sends the data to the Unix kernel and then to the program running such as the shell there is not reason we would see it.  Does the Shell send a copy back of what it received?  No it does not.  Rather typically the UNIX kernel is configured, by default, to send back a copy of what it receives on a specific terminal connection back to the terminal itself.  This is setting is called terminal ECHO mode.  So by default even before the Kernel sends data up to the destination program  it sends a copy back to the terminal so that the user can see what they typed.  This includes the `\n` that the user pressed to indicate the end of the command line.  Hence in this default mode any data send back from the shells behavior will appear on a new-line.  Programs can ask the kernel to disable this setting to have more control of what the users sees.  As a matter of fact like many things in Unix there is a command you can issue to have the kernel adjust the setting for you terminal connect including turning off the echo behavior (`stty -echo` to turn off and `stty echo` to turn back on).  Doing so lets you observe what is really being sent back in response to your command and not what the kernel automatically generated as echo data.  Note things might get confusing quickly if you turn echo off.   But Unix is all about understanding how things work and have the power to control them.  If you want to know more try `man stty` but be warned you will find that communications on a terminal is actually quite a bit more complex than it might seem -- there is a lot of history and skeletons buried here and the kernel is *cooking* the communications quite a bit ;-)

> What to have some fun.  Believe it or not we used to develop and play games on ASCII Terminals.  If you want to get an idea what cost some people their degrees try `umoria`